In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
import scipy.sparse
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE


In [2]:
df = pd.read_csv('merged_df_full_profile.csv')

C:\Users\Jainish\AppData\Local\Temp\ipykernel_11020\1616235069.py:1: DtypeWarning: Columns (137,139,141,142,143,144,145,146,147,148,149) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_df_full_profile.csv')


In [3]:
null_counts = df['Skills'].isnull().sum()

print(null_counts)

81291


In [4]:
print(df.columns)

Index(['Linkedin_url', 'target', 'profile_url', 'First Nam', 'Middle Nam',
       'Surnam', 'connections_x', 'Summary', 'Skills', 'title_1',
       ...
       'edu 9 dat', 'edu 9 degrees', 'edu 9 descriptio', 'edu 10 titl',
       'edu 10 dat', 'edu 10 degrees', 'edu 10 descriptio', 'merged_degrees',
       'merged_edu', 'highest_edu'],
      dtype='object', length=153)


In [5]:
df['Word_Count'] = df['Job_Description_1'].apply(lambda text: len(str(text).split()) if pd.notnull(text) else 0)
median_word_count = df['Word_Count'].median()
df['Word_Count'] = df['Word_Count'].replace(0, median_word_count)


In [6]:
df['summary_is_null'] = df['Job_Description_1'].isnull().astype(int)
df['skills_is_null'] = df['Skills'].isnull().astype(int)

In [7]:
df['skills_is_null'].value_counts()

0    104217
1     81291
Name: skills_is_null, dtype: int64

In [8]:
null_counts_target = df[['profile_url','Job_Description_1', 'Summary', 'Skills', 'Word_Count']].isnull().sum()
print("Null counts")
print(null_counts_target)

Null counts
profile_url               0
Job_Description_1    119520
Summary               56893
Skills                81291
Word_Count                0
dtype: int64


### Data Pre-processiong

In [9]:
import nltk
from nltk.corpus import stopwords
nltk.download('omw-1.4')
# Download stopwords and WordNet data from NLTK
nltk.download('stopwords')
nltk.download('wordnet')


def preprocess_text(text):
    if pd.isnull(text):  # Check if the text is null
        return ''
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and numbers
    text = re.sub(r"[^a-zA-Z]", " ", text)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words("english"))  # Use set(stopwords) instead of `stopwords`
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join the tokens back into a single string
    processed_text = " ".join(tokens)
    
    return processed_text

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Jainish\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jainish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jainish\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
def preprocess_skills(skills):
    if pd.isnull(skills):  # Check if the skills are null
        return ''

    skills = skills.lower()  # Convert skills to lowercase

    # Split the skills by commas
    skill_list = skills.split(',')

    # Process each individual skill
    processed_skills = []
    for skill in skill_list:
        # Remove leading/trailing whitespace
        skill = skill.strip()

        # Replace space with underscore
        skill = skill.replace(' ', '_')

        # Remove special characters and numbers
        skill = re.sub(r"[^a-zA-Z]", " ", skill)

        # Tokenize the skill
        tokens = word_tokenize(skill)

        # Remove stopwords
        stop_words = set(stopwords.words("english"))
        tokens = [token for token in tokens if token not in stop_words]

        # Lemmatize the tokens
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

        # Join the tokens back into a single string
        processed_skill = "_".join(tokens)
        processed_skills.append(processed_skill)

    # Join the processed skills back into a single string
    processed_text = ", ".join(processed_skills)

    return processed_text

In [11]:
df_selected = df[['profile_url','Job_Description_1', 'Summary', 'Skills', 'Word_Count', 'summary_is_null', 'skills_is_null', 'target']]

df_selected['Job_Description_1'] = df_selected['Job_Description_1'].apply(preprocess_text)
df_selected['Summary'] = df_selected['Summary'].apply(preprocess_text)

C:\Users\Jainish\AppData\Local\Temp\ipykernel_11020\3298440275.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Job_Description_1'] = df_selected['Job_Description_1'].apply(preprocess_text)
C:\Users\Jainish\AppData\Local\Temp\ipykernel_11020\3298440275.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Summary'] = df_selected['Summary'].apply(preprocess_text)


In [12]:
df_selected['Skills'] = df_selected['Skills'].apply(preprocess_skills)

X = df_selected[['profile_url','Job_Description_1', 'Summary', 'Skills', 'Word_Count', 'summary_is_null', 'skills_is_null']]
y = df_selected['target']

C:\Users\Jainish\AppData\Local\Temp\ipykernel_11020\1296174013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Skills'] = df_selected['Skills'].apply(preprocess_skills)


In [25]:
# Create TF-IDF vectorizer for Job_Description_1
tfidf_job_desc = TfidfVectorizer(max_features=10,use_idf=False)
X_job_desc_tfidf = tfidf_job_desc.fit_transform(X['Job_Description_1'])

# Get the feature names for Job_Description_1 and modify them
job_desc_feature_names = ['job_desc_' + name for name in tfidf_job_desc.get_feature_names_out()]

# Create TF-IDF vectorizer for Summary
tfidf_summary = TfidfVectorizer(max_features=10,use_idf=False)
X_summary_tfidf = tfidf_summary.fit_transform(X['Summary'])

# Get the feature names for Summary and modify them
summary_feature_names = ['summary_' + name for name in tfidf_summary.get_feature_names_out()]

# Create TF-IDF vectorizer for Skills
tfidf_skills = TfidfVectorizer(max_features=10, use_idf=False)
X_skills_tfidf = tfidf_skills.fit_transform(X['Skills'])
skills_feature_names = ['skills_' + name for name in tfidf_skills.get_feature_names_out()]

# Combine the TF-IDF features
X_combined = pd.DataFrame.sparse.from_spmatrix(scipy.sparse.hstack([X_job_desc_tfidf, X_summary_tfidf, X_skills_tfidf]), columns=job_desc_feature_names + summary_feature_names + skills_feature_names)

# Apply Variance Threshold Feature Selection on X_combined
variance_threshold = VarianceThreshold(threshold=0.01)
X_selected = variance_threshold.fit_transform(X_combined)

# Get the selected feature names
selected_feature_indices = variance_threshold.get_support(indices=True)
selected_feature_names = [X_combined.columns[i] for i in selected_feature_indices]

# Create a DataFrame with the selected features and Word_Count
X_selected_df = X_combined[selected_feature_names].copy()
X_selected_df['Word_Count'] = X['Word_Count']
X_selected_df['summary_is_null'] = X['summary_is_null']
X_selected_df['skills_is_null'] = X['skills_is_null']
X_selected_df['profile_url'] = X['profile_url']


X_selected_df.to_csv('text_data.csv', index=False)

In [26]:
X_selected_df = X_selected_df.drop('profile_url', axis=1)

In [27]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_selected_df, y, train_size=0.75)

In [28]:
X_selected_df.head()

,job_desc_application,job_desc_data,job_desc_design,job_desc_development,job_desc_project,job_desc_service,job_desc_software,job_desc_system,job_desc_team,job_desc_using,...,skills_java,skills_javascript,skills_linux,skills_mysql,skills_python,skills_software_development,skills_sql,Word_Count,summary_is_null,skills_is_null
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.00000,1.000000,0.000000,0,1,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.00000,0.707107,0.000000,4,0,0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.577350,0.0,0.57735,0.577350,0.000000,116,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0,1,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.447214,0.447214,0.447214,0.0,0.00000,0.000000,0.447214,2,0,0


In [29]:
X_selected_df.columns

Index(['job_desc_application', 'job_desc_data', 'job_desc_design',
       'job_desc_development', 'job_desc_project', 'job_desc_service',
       'job_desc_software', 'job_desc_system', 'job_desc_team',
       'job_desc_using', 'summary_application', 'summary_data',
       'summary_development', 'summary_experience', 'summary_software',
       'summary_system', 'summary_team', 'summary_technology', 'summary_web',
       'summary_year', 'skills_agile_methodology', 'skills_cs', 'skills_html',
       'skills_java', 'skills_javascript', 'skills_linux', 'skills_mysql',
       'skills_python', 'skills_software_development', 'skills_sql',
       'Word_Count', 'summary_is_null', 'skills_is_null'],
      dtype='object')

In [30]:
df['target'].value_counts()

0    166810
1     18698
Name: target, dtype: int64

In [31]:
from sklearn.naive_bayes import MultinomialNB

# List of models to evaluate
models = {
    "XGBoost": XGBClassifier(),
    "LightGBM": LGBMClassifier(),
    "Multinomial Naive Bayes": MultinomialNB(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
}

# Evaluate each model
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Predict on the test data
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"{model_name} Accuracy: {accuracy:.4f}")
    
    # Generate classification report
    classification_rep = classification_report(y_test, y_pred)
    print()
    print(f"{model_name} Classification Report:")
    print(classification_rep)


XGBoost Accuracy: 0.9005

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.90      1.00      0.95     41788
           1       0.24      0.00      0.01      4589

    accuracy                           0.90     46377
   macro avg       0.57      0.50      0.48     46377
weighted avg       0.84      0.90      0.85     46377

LightGBM Accuracy: 0.9010

LightGBM Classification Report:
              precision    recall  f1-score   support

           0       0.90      1.00      0.95     41788
           1       0.00      0.00      0.00      4589

    accuracy                           0.90     46377
   macro avg       0.45      0.50      0.47     46377
weighted avg       0.81      0.90      0.85     46377

Multinomial Naive Bayes Accuracy: 0.8247


C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(



Multinomial Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.90      0.90     41788
           1       0.15      0.16      0.16      4589

    accuracy                           0.82     46377
   macro avg       0.53      0.53      0.53     46377
weighted avg       0.83      0.82      0.83     46377



C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


K-Nearest Neighbors Accuracy: 0.8908

K-Nearest Neighbors Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.99      0.94     41788
           1       0.12      0.02      0.03      4589

    accuracy                           0.89     46377
   macro avg       0.51      0.50      0.49     46377
weighted avg       0.82      0.89      0.85     46377



...

C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 i

Gradient Boosting Accuracy: 0.9011

Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       0.90      1.00      0.95     41788
           1       0.00      0.00      0.00      4589

    accuracy                           0.90     46377
   macro avg       0.45      0.50      0.47     46377
weighted avg       0.81      0.90      0.85     46377



C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


AdaBoost Accuracy: 0.9011

AdaBoost Classification Report:
              precision    recall  f1-score   support

           0       0.90      1.00      0.95     41788
           1       0.00      0.00      0.00      4589

    accuracy                           0.90     46377
   macro avg       0.45      0.50      0.47     46377
weighted avg       0.81      0.90      0.85     46377



C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0],
    'fit_prior': [True, False]
}

# Create the classifier
nb = MultinomialNB()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=nb, param_grid=param_grid, cv=5, scoring='accuracy')

# Perform the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\ana

C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\ana

Best Parameters: {'alpha': 2.0, 'fit_prior': True}
Best Score: 0.8233391306619907


C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [33]:
model = MultinomialNB(alpha=2, fit_prior=True)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
classification_rep = classification_report(y_test, y_pred)
print(classification_rep)

C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.91      0.90      0.90     41788
           1       0.15      0.16      0.16      4589

    accuracy                           0.82     46377
   macro avg       0.53      0.53      0.53     46377
weighted avg       0.83      0.82      0.83     46377



C:\Users\Jainish\anaconda3\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
